Notebook description: <br>

This notebook is created to analysis the kinematics data obtained after running the tracking code. <br>

Most of the tracking and analysis codes are written by Duncan and pieced together in this notebook. <br>

This one is specific to analysing eye convergence.

### Import experiment (tracked and with kinematics data).

In [ ]:
from behavior_analysis.experiment import BehaviorExperiment
import pandas as pd
# Open experiment
experiment = BehaviorExperiment.open(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\uv_trial")
print(experiment)
# Open video and bout info
video_info = pd.read_csv(experiment.directory.joinpath('video_data.csv'), dtype={'ID': str, 'code': str})
bouts_df = pd.read_csv(experiment.subdirs["analysis"].joinpath('bouts.csv'),
                        dtype={'ID': str, 'code': str})

### Data information

- Fish ID in the format 'YYYYMMDDNN' (year, month, day, zero-padded fish number; e.g. fish_1 on 2025_05_29 would be 20250529) <br>

- Video code in the format 'IDHHMMSS' (fish_ID, hour, minute, second) <br>

- Number of rows == number of frames in the video

#### Description of data in the "kinematics/code + .csv"
| Column name | Column data description |
| --- | --- |
| 'k0' - 'k(n-1)' | angle of tangents between n successive tail points |
| 'tip' | the average curvature (tail angle) over the last 20% of the tail|
| 'length' | the length of the tail (useful for finding tracking errors) |
| 'left' | the angle of the left eye relative to the heading |
| 'right' | the angle of the right eye relative to the heading |
| 'speed' | the instantaneous speed in each frame |
| 'angular_velocity' | the instantaneous angular velocity in each frame |
| 'tracked' | whether kinematic data exists from the frame |

## Eye convergence analysis

#### Before running Duncan's analysis codes, let's check out what the eye angle looks like by simply plotting them out!

In [ ]:
# load kinematics data
# example data info:
# fish: progeny of mafaa:QF2, ath5:Cre, QUAS:switchNTR outX (fish_3)
# condition: -NTR, +UV (no mafaa+ RGC ablation)
# videos: 250fps (unstable fps for this test recording), 2.5min each, 12 recordings (30min) in total

# 2nd video of the trial, no paramecia
test_kinematics_df1 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060603/2025060603163529.777.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
# 7th video of the trial, with paramecia
test_kinematics_df2 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060603/2025060603165036.654.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
# last video of the trial, with paramecia
test_kinematics_df3 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060603/2025060603170312.514.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
print(test_kinematics_df1, test_kinematics_df2)
test_kinematics_df3

In [ ]:
# try plotting with seaborn
import seaborn as sns
# set figure size
sns.set_theme(rc={"figure.figsize": (20,5)})

In [ ]:
# check out the speed of the fish
sns.lineplot(data=test_kinematics_df1, x=test_kinematics_df1.index, y="speed") # in blue
sns.lineplot(data=test_kinematics_df2, x=test_kinematics_df2.index, y="speed") # in orange
sns.lineplot(data=test_kinematics_df3, x=test_kinematics_df3.index, y="speed") # in green

In [ ]:
# check out the angular velocity of the fish
sns.lineplot(data=test_kinematics_df1, x=test_kinematics_df1.index, y="angular_velocity") # in blue
sns.lineplot(data=test_kinematics_df2, x=test_kinematics_df2.index, y="angular_velocity") # in orange
sns.lineplot(data=test_kinematics_df3, x=test_kinematics_df3.index, y="angular_velocity") # in green

In [ ]:
# melt the df for plotting left and right eye angles at the same scale
test_kinematics_df1 = test_kinematics_df1.reset_index().rename(columns={'index': 'frame'}).melt( # reset_index and rename put the frame number (the original index) in a separate column
    id_vars= ['speed','angular_velocity','frame'], # these columns will be kept the same
    value_vars= ['left_angle','right_angle'], # these columns will be melted
    var_name= "direction", # the column name 'left_angle' and 'right_angle' will now label the angle degree in a separate column
    value_name= "angle (degree)" # the values of the 'left_angle' and 'right_angle' columns are now in one columns
)
test_kinematics_df1

In [ ]:
# description same as above but for the second df
test_kinematics_df2 = test_kinematics_df2.reset_index().rename(columns={'index': 'frame'}).melt(
    id_vars= ['speed','angular_velocity','frame'],
    value_vars= ['left_angle','right_angle'],
    var_name= "direction",
    value_name= "angle (degree)"
)
test_kinematics_df2

In [ ]:
# description same as above but for the third df
test_kinematics_df3 = test_kinematics_df3.reset_index().rename(columns={'index': 'frame'}).melt( 
    id_vars= ['speed','angular_velocity','frame'],
    value_vars= ['left_angle','right_angle'],
    var_name= "direction",
    value_name= "angle (degree)"
)
test_kinematics_df3

In [ ]:
# at the beginning of trial, before introduction of paramecia
sns.lineplot(data=test_kinematics_df1, x='frame', y="angle (degree)", hue="direction", legend="auto")

In [ ]:
# in the middle of trial, after introduction of paramecia
sns.lineplot(data=test_kinematics_df2, x='frame', y="angle (degree)", hue="direction", legend="auto")

In [ ]:
# at the end of the trial, after introduction of paramecia
sns.lineplot(data=test_kinematics_df3, x='frame', y="angle (degree)", hue="direction", legend="auto")

In [ ]:
# load kinematics data
# example data info:
# fish: progeny of mafaa:QF2, ath5:Cre, QUAS:switchNTR outX (fish_2)
# condition: -NTR, -UV (no mafaa+ RGC ablation)
# videos: 250fps (unstable fps for this test recording), 2.5min each, 12 recordings (30min) in total

# 2nd video of the trial, no paramecia
test_kinematics_df1 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060602/2025060602155607.538.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
# 6th video of the trial, with paramecia
test_kinematics_df2 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060602/2025060602160612.354.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
# 2nd last video of the trial, with paramecia
test_kinematics_df3 = pd.read_csv(experiment.subdirs['kinematics'].joinpath("2025060602/2025060602162119.139.csv")).get(["speed", "angular_velocity","left_angle","right_angle"])
print(test_kinematics_df1, test_kinematics_df2)
test_kinematics_df3

In [ ]:
# check out the speed of the fish
sns.lineplot(data=test_kinematics_df1, x=test_kinematics_df1.index, y="speed") # in blue
sns.lineplot(data=test_kinematics_df2, x=test_kinematics_df2.index, y="speed") # in orange
sns.lineplot(data=test_kinematics_df3, x=test_kinematics_df3.index, y="speed") # in green

In [ ]:
# check out the angular velocity of the fish
sns.lineplot(data=test_kinematics_df1, x=test_kinematics_df1.index, y="angular_velocity") # in blue
sns.lineplot(data=test_kinematics_df2, x=test_kinematics_df2.index, y="angular_velocity") # in orange
sns.lineplot(data=test_kinematics_df3, x=test_kinematics_df3.index, y="angular_velocity") # in green

In [ ]:
# melt the df for plotting left and right eye angles at the same scale
test_kinematics_df1 = test_kinematics_df1.reset_index().rename(columns={'index': 'frame'}).melt( # reset_index and rename put the frame number (the original index) in a separate column
    id_vars= ['speed','angular_velocity','frame'],
    value_vars= ['left_angle','right_angle'],
    var_name= "direction",
    value_name= "angle (degree)"
)

test_kinematics_df2 = test_kinematics_df2.reset_index().rename(columns={'index': 'frame'}).melt( # reset_index and rename put the frame number (the original index) in a separate column
    id_vars= ['speed','angular_velocity','frame'],
    value_vars= ['left_angle','right_angle'],
    var_name= "direction",
    value_name= "angle (degree)"
)
test_kinematics_df3 = test_kinematics_df3.reset_index().rename(columns={'index': 'frame'}).melt( # reset_index and rename put the frame number (the original index) in a separate column
    id_vars= ['speed','angular_velocity','frame'],
    value_vars= ['left_angle','right_angle'],
    var_name= "direction",
    value_name= "angle (degree)"
)

test_kinematics_df3 # check if it's melted

In [ ]:
# at the beginning of trial, before introduction of paramecia
sns.lineplot(data=test_kinematics_df1, x='frame', y="angle (degree)", hue="direction", legend="auto")

In [ ]:
# in the middle of trial, after introduction of paramecia
sns.lineplot(data=test_kinematics_df2, x='frame', y="angle (degree)", hue="direction", legend="auto")

In [ ]:
# at the end of the trial, after introduction of paramecia
sns.lineplot(data=test_kinematics_df3, x='frame', y="angle (degree)", hue="direction", legend="auto")

#### Eye convergence analysis (code scavanged and modified from Duncan's files) <br>

Below spits out: <br>

- Convergence threshold <br>

- Convergence score (the proportion of frames above the threshold) <br>

- Histogram of eye angles distribution (saved in analysis/eye_convergence/plots)

In [3]:
from behavior_analysis.experiment import BehaviorExperiment
from behavior_analysis.analysis.bouts import BoutData
from behavior_analysis.analysis import eye_tracking_data
from behavior_analysis.utilities.manage_files_helpers import create_folder
from pathlib import Path
import os

In [4]:
from behavior_analysis.experiment import  BehaviorExperiment
import pandas as pd
# Open experiment
experiment = BehaviorExperiment.open(r"C:\Users\manyung.ng\Documents\behaviour_analysis\behavior_analysis_tracking\uv_trial")
print(experiment)

name: uv_trial
date: 2025-06-25
animal_data: fish_data.csv
video_data: video_data.csv
mask_data: mask_data.csv
bout_detection: {'threshold': 0.02, 'winsize': 0.05}



this chunk below now works

In [5]:
output_directory = create_folder(experiment.subdirs['analysis'], 'eye_convergence')
convergence_scores_path = os.path.join(output_directory, 'convergence_scores.csv')

# create eye angle data (left, right, converge, in degrees)
eye_tracking = eye_tracking_data.EyeTrackingData.from_experiment(experiment)

plots_directory = create_folder(output_directory, 'plots')


IMPORTING EYE ANGLE DATA


2025060601
----------
Importing data (12 files)
1
2
3
4
5
6
7
8
9
10
11
12

done!


2025060602
----------
Importing data (12 files)
1
2
3
4
5
6
7
8
9
10
11
12

done!


2025060603
----------
Importing data (12 files)
1
2
3
4
5
6
7
8
9
10
11
12

done!



In [6]:
convergence_scores = eye_tracking.calculate_convergence_scores(save_plots_to=plots_directory)
convergence_scores.to_csv(convergence_scores_path, index=False)


CALCULATING CONVERGENCE SCORES


2025060601
----------
Performing kernel density estimation...
done!
Eye convergence threshold: 51.0
Eye convergence score: 0.11859517028350687


2025060602
----------
Performing kernel density estimation...
done!
No local minimum within limits!
Eye convergence threshold: 50.0
Eye convergence score: 0.04044029992089754


2025060603
----------
Performing kernel density estimation...
done!
Eye convergence threshold: 54.0
Eye convergence score: 0.2703731200625942



### Bout classification including eye convergence data

In [8]:
bouts_path = os.path.join(experiment.subdirs['analysis'], 'bouts.csv')
bouts_df = pd.read_csv(bouts_path, index_col=0, dtype={'ID': str, 'video_code': str})

eye_convergence_directory = os.path.join(experiment.subdirs['analysis'], 'eye_convergence')
convergence_scores_path = os.path.join(eye_convergence_directory, 'convergence_scores.csv')

convergence_scores = pd.read_csv(convergence_scores_path, dtype={'ID': str})

frame_rate = 250.
window = int(0.02 * 250)

# Import bout data
bouts = bouts = BoutData.from_metadata(bouts_df, experiment.subdirs['kinematics'], tail_only=False)

print_heading('CLASSIFYING BOUTS')
convergence_states = np.empty((len(bouts_df), 4))
i = 0
for idx, fish_info in convergence_scores.iterrows():
    print(fish_info.ID)
    for bout in bouts.list_bouts(IDs=[fish_info.ID]):
        bout_convergence = np.degrees(bout['right'] - bout['left'])
        convergence_start = bout_convergence[:window].mean()
        convergence_end = bout_convergence[-window:].mean()
        convergence_states[i, :2] = np.array([convergence_start, convergence_end])
        convergence_states[i, 2:] = (np.array([convergence_start, convergence_end]) >= fish_info.threshold)
        i += 1
assert i == len(convergence_states), 'Incorrect number of bouts!'
np.save(os.path.join(eye_convergence_directory, 'convergence_states.npy'), convergence_states)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')